In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from Network import KANLayer3 as KANLayer
import tensorflow_datasets as tfds

D:\Software\anaconda3\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.12.2 when it was built against 1.12.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
class Model(tf.Module):
    def __init__(self, ParameterLayers, seed = None, name = ""):
        super().__init__(name=name)
        self.KANLayers = []
        i = 0
        for param in ParameterLayers:
            self.KANLayers.append(KANLayer(param[0], param[1], GInterval = param[2], p=param[3], seed = seed, name = ""+"_Layer"+str(i)))
            i+=1

    def __call__(self, X):
        X = tf.keras.layers.Flatten()(X)
        for kan in self.KANLayers:
            X = kan(X)
        return tf.keras.activations.softmax(X, axis = 1)
        #return X

In [3]:
Params = [[784, 30, 4, 3],
         [30, 20, 10, 3],
         [20, 20, 20, 3], 
         [20, 10, 30, 3]]

In [4]:
((train_X, train_Y), (test_X, test_Y)) = tf.keras.datasets.mnist.load_data(
    path='mnist.npz'
)

In [5]:
train_epoch_X = []
train_epoch_Y = []
N = 10000
i = 0
while((i+N) <= 60000):
    a = train_Y[i:(i+N)]
    b = np.zeros((a.size, a.max() + 1))
    b[np.arange(a.size), a] = 1
    train_epoch_X.append(train_X[i:(i+N)]/255)
    train_epoch_Y.append(b)
    i+=N

In [6]:
model = Model(Params, seed = 100, name = "Mod1")

In [7]:
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [ ]:
i = 0
epochs = 20
for _ in range(epochs):
    for X, Y in zip(train_epoch_X, train_epoch_Y):
        print("epoch: ",i, end = "\t")
        with tf.GradientTape() as tape:
            Y_pred = model(X)
            mse_loss = loss(Y, Y_pred)

        
        Y_pred_vals = tf.argmax(Y_pred, axis = 1)
        Y_true_vals = tf.argmax(Y, axis = 1)
        Acc = tf.math.reduce_sum(tf.where(tf.math.equal(Y_true_vals, Y_pred_vals), 1, 0)) * 100 / N
        print("Acc: ", Acc.numpy(),"%", end = "\t")
        
        tf.print(np.sum(mse_loss)/N)
        dY_dM = tape.gradient(mse_loss, model.trainable_variables)


        #Correct Nan Values and set them to 0
        grads = []
        for grad in dY_dM:
            grads.append(tf.where(tf.math.is_nan(grad), tf.zeros_like(grad), grad))
            if(tf.reduce_any(tf.math.is_nan(grad) == True)):
                print("NAN_Exists")
        print()
                
        optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
        i+=1

epoch:  0	Acc:  11.25 %	0.001895323371887207

epoch:  1	Acc:  10.01 %	0.0009276450157165528

epoch:  2	Acc:  5.56 %	0.0004532963275909424

epoch:  3	Acc:  8.04 %	0.0002769891023635864

epoch:  4	